In [1]:
from fastai.text.all import *
import polars as pl

In [2]:
pl.Config.set_fmt_str_lengths(200);

# Data

In [3]:
labels_dict = {0:'sadness', 1:'joy', 2:'love', 3:'anger', 4:'fear', 5:'surprise'}
df = (
    pl.concat([
        pl.scan_csv('emotions-dataset/training.csv').with_columns(is_valid=False),
        pl.scan_csv('emotions-dataset/validation.csv').with_columns(is_valid=True),
    ])
    .with_columns(pl.col.label.replace_strict(labels_dict).alias('label_name'))
    .collect()
)
print(df.shape)
df.head()

(18000, 4)


text,label,is_valid,label_name
str,i64,bool,str
"""i didnt feel humiliated""",0,false,"""sadness"""
"""i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake""",0,false,"""sadness"""
"""im grabbing a minute to post i feel greedy wrong""",3,false,"""anger"""
"""i am ever feeling nostalgic about the fireplace i will know that it is still on the property""",2,false,"""love"""
"""i am feeling grouchy""",3,false,"""anger"""


In [7]:
dls = TextDataLoaders.from_df(df.to_pandas(), text_col='text', label_col='label_name', valid_col='is_valid')
dls.show_batch(max_n=3)

,text,category
0,xxbos i feel in my bones like nobody cares if i m here nobody cares if i m gone here i am again saying i m feeling so lonely people either say its ok to be alone or just go home it xxunk me and i do nt know why it does nt mean i do nt try i try and try but people just treat me like i m a xxunk,sadness
1,xxbos i might go out of existance i smile pick up my pen and fill the page with the things that you say the thoughts you obtain the moments you xxunk far away its cause you re going insane and suddenly i m left afraid because i m not feeling that way instead i m amazed why you got ta be that way,surprise
2,xxbos i feel rotten all week because i hardly ever see you that s why i wrote this hopeless song i ve never been in love with a girl like you before darling come with me such a wonderful thing has never happened to me before you re the only one who touched my heart it s all a question of courage,sadness


In [8]:
L(dls.numericalize.vocab)

(#5600) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','i','feel','and','to','the','a','that','feeling','of','my','in'...]

In [9]:
dls.categorize.vocab

['anger', 'fear', 'joy', 'love', 'sadness', 'surprise']

In [10]:
df[0]

text,label,is_valid,label_name
str,i64,bool,str
"""i didnt feel humiliated""",0,false,"""sadness"""


In [11]:
dls.train_ds[0]

(TensorText([  2,   9,  81,  36,  10, 654]), TensorCategory(4))

In [12]:
f'Label: {dls.categorize.vocab[dls.train_ds[0][1]]}'

'Label: sadness'

In [13]:
[dls.numericalize.vocab[o] for o in dls.train_ds[0][0]]

['xxbos', 'i', 'did', 'nt', 'feel', 'humiliated']

## Model

In [14]:
vocab_size = len(dls.numericalize.vocab)
vocab_size

5600

In [15]:
n_labels = len(dls.categorize.vocab)
n_labels

6

In [16]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.embs = nn.Embedding(vocab_size, 100, padding_idx=1)
        self.lstm = nn.LSTM(100, 20, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(20, 8),
            nn.ReLU(),
            nn.Linear(8, n_labels)
        )
    
    def forward(self, x, verbose=False):
        embs = self.embs(x)
        if verbose: print(f'embs: {embs.shape}')
        h,_ = self.lstm(embs)
        if verbose: print(f'h: {h.shape}')
        h = h[:,-1]
        if verbose: print(f'h[:,-1]: {h.shape}')
        out = self.fc(h)
        if verbose: print(f'out: {out.shape}')
        return out
    
model = Model().cuda()

In [17]:
model

Model(
  (embs): Embedding(5600, 100, padding_idx=1)
  (lstm): LSTM(100, 20, batch_first=True)
  (fc): Sequential(
    (0): Linear(in_features=20, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=6, bias=True)
  )
)

In [18]:
for i,(xb,yb) in enumerate(dls.train):
    print(xb.shape)
    if i==2: break

torch.Size([64, 72])
torch.Size([64, 19])
torch.Size([64, 11])


In [19]:
with torch.no_grad(): res = model(xb, verbose=True)

embs: torch.Size([64, 11, 100])
h: torch.Size([64, 11, 20])
h[:,-1]: torch.Size([64, 20])
out: torch.Size([64, 6])


In [20]:
learn = Learner(dls, model, metrics=accuracy)

## Train

In [21]:
learn.fit_one_cycle(10, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.557495,1.541906,0.390500,00:01
1,1.045136,0.887870,0.671000,00:01
2,0.427761,0.413745,0.864000,00:01
3,0.209701,0.236360,0.914000,00:01
4,0.122803,0.224680,0.912000,00:01
5,0.081514,0.218477,0.916000,00:01
6,0.055239,0.232407,0.922000,00:01
7,0.031727,0.247382,0.917000,00:01
8,0.021584,0.270350,0.921000,00:01
9,0.019629,0.273700,0.918500,00:01


In [53]:
learn.show_results()

,text,category,category_
0,xxbos i tell myself i do nt open my mouth and say what i really feel because i know i m a xxunk and i m smart enough to know i m a xxunk and i never ever know if what i m doing the choices i make are really what i want or need or even the right thing or if its the disease,joy,joy
1,xxbos i could nt hellip even when it made my heart ache to simply look at you hellip because i loved you so much and i knew you would never return my feelings hellip and i could nt bring myself to hate you for the idiotic xxunk you pulled in the other room either though i do ask that you do nt repeat it,sadness,sadness
2,xxbos i really want to watch it for the obvious romance reasons and i have a feeling like it would be a really funny kind of drama too i can also somewhat relate to the female main character who works with this boss who has a lot of pride and is a bit what you would like to call him as xxunk,surprise,surprise
3,xxbos i could go on and on right now about what we ve been through this year and what i ve learned what xxunk could do when and such but i wo nt because this would be a book and honestly i m not feeling fabulous today and xxunk has been dealing with a giant cold since thursday and we are wiped,joy,joy
4,xxbos i feel stressed out all the time i said and then i think about how people say stress causes cancer and i know it isn t true but i can t stop thinking that i need to relax or else my cancer will come back and then i get stressed out because i m stressed and it makes me feel worse,sadness,sadness
5,xxbos i just feel you so so don t be afraid i should hurt even more and pray again so i can find you again the more time passes the more it hurts i need you go back in time just one time forgive my sins if only i could turn things back this pain would be so so sloth,fear,fear
6,xxbos i know my xxunk is stronger than my behaviour over the weekend and i need to focus on the joy and health that all the great food i brought with me gives and how i could ve if i really wanted to indulge indulged in that great stuff i know its not the same but i would feel amazing,joy,joy
7,xxbos i have been out there over the last few weeks i experienced for the first time a feeling of loving the actual act of running of pushing my daughter in the xxunk of getting xxunk by my wife although this would happen if i was in top shape anyway of having cold air nail you in the face,love,love
8,xxbos i suggest you do though it might be hard cause it is a bit slow at times if you don t feel a bit of a tug at your heart or perhaps feel a tear xxunk in your tear xxunk i will declare that you are heartless and thus should be xxunk from the rest of the world,anger,anger


In [55]:
interp = Interpretation.from_learner(learn)

In [60]:
interp.plot_top_losses(5)

,input,target,predicted,probability,loss
0,xxbos i know exactly how she feels because i hated it so badly i got so depressed i was cutting myself when i got so low i started thinking about suicide i did run away to nyc the xxunk place from them where they would nt be able to find me,anger,sadness,0.9999885559082031,17.717792510986328
1,xxbos i like the three finger hands those simple details give it that xxunk feel again the paint choices while not terrible by any stretch of the imagination it doesn t blow me away and i would have liked to have seen these both in xxunk blue,sadness,love,0.9952428936958313,16.64166259765625
2,xxbos i don t always feel like i have amazing style and most days i choose comfort over anything else but there is one thing that i feel makes all the difference in how i feel about myself and that is makeup,surprise,joy,0.9999657869338989,13.931456565856934
3,xxbos i feel so amazing about taking this trip as i think ill finally be able to relax and feel comfortable at home and somehow just xxunk back into it,surprise,joy,0.9999949932098389,12.271406173706055
4,xxbos i am surprised that she is shocked by what i have said and begin to feel dismayed as she becomes increasingly sympathetic in her responses towards me,sadness,surprise,0.963080108165741,11.939552307128906


In [76]:
learn.predict('Write here some random phrase here', with_input=True)

('xxbos xxmaj write here some random phrase here',
 'love',
 tensor(3),
 tensor([0.1439, 0.0289, 0.3381, 0.4497, 0.0082, 0.0313]))